In [ ]:
from flask import Flask, jsonify, request
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///books.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)

class Book(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    book_name = db.Column(db.String(50))
    author = db.Column(db.String(50))
    publisher = db.Column(db.String(50))

    def __init__(self, book_name, author, publisher):
        self.book_name = book_name
        self.author = author
        self.publisher = publisher

@app.route('/books', methods=['GET'])
def get_all_books():
    books = Book.query.all()
    result = []
    for book in books:
        book_data = {}
        book_data['id'] = book.id
        book_data['book_name'] = book.book_name
        book_data['author'] = book.author
        book_data['publisher'] = book.publisher
        result.append(book_data)
    return jsonify(result)

@app.route('/books/<id>', methods=['GET'])
def get_book(id):
    book = Book.query.get(id)
    if not book:
        return jsonify({'error': 'Book not found'})
    book_data = {}
    book_data['id'] = book.id
    book_data['book_name'] = book.book_name
    book_data['author'] = book.author
    book_data['publisher'] = book.publisher
    return jsonify(book_data)

@app.route('/books', methods=['POST'])
def add_book():
    book_name = request.json['book_name']
    author = request.json['author']
    publisher = request.json['publisher']
    book = Book(book_name=book_name, author=author, publisher=publisher)
    db.session.add(book)
    db.session.commit()
    return jsonify({'message': 'Book added successfully'})

@app.route('/books/<id>', methods=['PUT'])
def update_book(id):
    book = Book.query.get(id)
    if not book:
        return jsonify({'error': 'Book not found'})
    book_name = request.json['book_name']
    author = request.json['author']
    publisher = request.json['publisher']
    book.book_name = book_name
    book.author = author
    book.publisher = publisher
    db.session.commit()
    return jsonify({'message': 'Book updated successfully'})

@app.route('/books/<id>', methods=['DELETE'])
def delete_book(id):
    book = Book.query.get(id)
    if not book:
        return jsonify({'error': 'Book not found'})
    db.session.delete(book)
    db.session.commit()
    return jsonify({'message': 'Book deleted successfully'})

if __name__ == '__main__':
    app.run(debug=True)
